**Hello, welcome to the preprocessing notebook of our data science project: predicting successful startup!**

We will try to include as much details as possible with markdowns :)

# Data Import

In [383]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [384]:
data = pd.read_csv('/Users/danyhersco/startups.csv')

# Data Exploration

In [385]:
data.shape

(314486, 19)

Let's quickly drop real duplicates (see below for "false duplicates"):

In [386]:
data = data.drop_duplicates()
data.shape

(312667, 19)

We extracted the dataset from DBeaver, after running an SQL query. As a result of multiple `LEFT JOIN` to include `industry` and `technology` features, many startups have duplicates.

Consequently, let's check for the number of unique values for each feature of our dataframe.

In [387]:
data.describe(include='all')

,id,name,website,short_description,ipo_status,founded_on,went_public_on,exited_on,num_funding_rounds,last_equity_funding_type,last_equity_funding_total,last_funding_at,headquartersCountry,headquartersRegion,employeeCount,industry_name,technology_name,announcedOn,moneyRaised
count,312667,312667,309473,312664,312667,312667,5360,21964,312667.000000,287213,235134,312667,311528,311485,300442.000000,303549,297730,312016,222598
unique,56667,55665,54614,56142,3,1083,507,1370,NaN,18,15347,2905,162,1392,NaN,40,8,3360,34767
top,4f97ceac-342b-425d-a947-f03c48012b20,Arthur Intelligence,https://www.goarthur.ai/,Limited information available,private,2016-01-01,2021-11-11,2021-02-19,NaN,seed,"{""currency"":""USD"",""amount"":100000000,""amountUS...",2019-01-01,US,California,NaN,Health Care,Software,2019-01-01 00:00:00.000,"{""amount"": 10000000, ""currency"": ""USD"", ""amoun..."
freq,216,216,216,597,307633,67127,114,146,NaN,107046,4766,2877,137563,49381,NaN,29218,163702,4335,5642
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.406656,NaN,NaN,NaN,NaN,NaN,62.002882,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.419148,NaN,NaN,NaN,NaN,NaN,444.244820,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,6.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,31.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,31.000000,NaN,NaN,NaN,NaN


We have **55665** unique startups in our dataset. The next step will be to remove all duplicates while treating the non-duplicated columns.

Here is how our dataset look like:

In [388]:
data.head()

,id,name,website,short_description,ipo_status,founded_on,went_public_on,exited_on,num_funding_rounds,last_equity_funding_type,last_equity_funding_total,last_funding_at,headquartersCountry,headquartersRegion,employeeCount,industry_name,technology_name,announcedOn,moneyRaised
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Video,Science and Engineering,2018-05-30 00:00:00.000,NaN
1,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Video,Hardware,2018-05-30 00:00:00.000,NaN
2,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Computer Hardware,Science and Engineering,2018-05-30 00:00:00.000,NaN
3,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Computer Hardware,Hardware,2018-05-30 00:00:00.000,NaN
4,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Media and Entertainment,Science and Engineering,2018-05-30 00:00:00.000,NaN


# Remove the duplicates through encoding

This problem will be challenging as there is not really fully duplicated rows. As we said above, it is just a result of the `LEFT JOIN` that multiplicated the rows for one company (several industries, several technologies, and several funding rounds). We have to one hot encode the latter features, which is what we will be doing in this section.

At the end of this section, we should reach a shape of `(55665, >18)`.

In [389]:
data.head()

,id,name,website,short_description,ipo_status,founded_on,went_public_on,exited_on,num_funding_rounds,last_equity_funding_type,last_equity_funding_total,last_funding_at,headquartersCountry,headquartersRegion,employeeCount,industry_name,technology_name,announcedOn,moneyRaised
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Video,Science and Engineering,2018-05-30 00:00:00.000,NaN
1,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Video,Hardware,2018-05-30 00:00:00.000,NaN
2,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Computer Hardware,Science and Engineering,2018-05-30 00:00:00.000,NaN
3,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Computer Hardware,Hardware,2018-05-30 00:00:00.000,NaN
4,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,NaN,2018-05-30,CN,Shanghai,31.0,Media and Entertainment,Science and Engineering,2018-05-30 00:00:00.000,NaN


##  `industry` encoding

In [442]:
data_ind = data[['id', 'industry_name']].drop_duplicates()
data_ind.head()

,id,industry_name
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Video
2,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Computer Hardware
4,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Media and Entertainment
6,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Consumer Electronics
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,Privacy and Security


In [443]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
data_ind[encoder.get_feature_names_out()] = encoder.fit_transform(data_ind[['industry_name']])

In [444]:
data_ind.head()

,id,industry_name,industry_name_Advertising,industry_name_Agriculture and Farming,industry_name_Clothing and Apparel,industry_name_Commerce and Shopping,industry_name_Community and Lifestyle,industry_name_Computer Hardware,industry_name_Consumer Electronics,industry_name_Consumer Goods,...,industry_name_Privacy and Security,industry_name_Professional Services,industry_name_Real Estate and Construction,industry_name_Sales and Marketing,industry_name_Software,industry_name_Sports,industry_name_Transportation,industry_name_Travel and Tourism,industry_name_Video,industry_name_nan
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Video,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Computer Hardware,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Media and Entertainment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Consumer Electronics,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,Privacy and Security,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [445]:
for column in data_ind.columns[2:]:
    data_ind.rename(columns={column: column[14:]}, inplace=True)

In [446]:
data_ind.head()

,id,industry_name,Advertising,Agriculture and Farming,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Computer Hardware,Consumer Electronics,Consumer Goods,...,Privacy and Security,Professional Services,Real Estate and Construction,Sales and Marketing,Software,Sports,Transportation,Travel and Tourism,Video,nan
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Video,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Computer Hardware,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Media and Entertainment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Consumer Electronics,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,Privacy and Security,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [447]:
data_ind = data_ind.groupby('id').max()
data_ind.head()

,industry_name,Advertising,Agriculture and Farming,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Computer Hardware,Consumer Electronics,Consumer Goods,Content and Publishing,...,Privacy and Security,Professional Services,Real Estate and Construction,Sales and Marketing,Software,Sports,Transportation,Travel and Tourism,Video,nan
id,,,,,,,,,,,,,,,,,,,,,
00003f34-ac89-4090-b592-30f2b70ff5a4,Environment and Sustainability,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,Community and Lifestyle,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,Health Care,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003088d-8c0b-46f6-910a-6dea65777ac4,Transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0004d27e-6d98-4f29-a708-b989e3e38f81,Health Care,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [448]:
data_ind.reset_index(inplace=True)
data_ind.drop(columns=['nan', 'industry_name'], inplace=True)

In [449]:
data_ind.shape

(56667, 41)

##  `technology` encoding

In [453]:
data_tec = data[['id', 'technology_name']].drop_duplicates()
data_tec.head()

,id,technology_name
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Science and Engineering
1,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Hardware
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,Software
10,59b8a226-eb24-4e7b-851d-924e0385d01c,Software
14,f786e92e-7ac1-4035-9644-de30b8d8b6a5,Biotechnology


In [454]:
encoder2 = OneHotEncoder(sparse=False)
data_tec[encoder2.get_feature_names_out()] = encoder2.fit_transform(data_tec[['technology_name']])

In [455]:
data_tec.head()

,id,technology_name,technology_name_AR and VR,technology_name_Artificial Intelligence,technology_name_Biotechnology,technology_name_BlockChain,technology_name_Hardware,technology_name_Science and Engineering,technology_name_Software,technology_name_Sustainability,technology_name_nan
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Science and Engineering,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Hardware,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,59b8a226-eb24-4e7b-851d-924e0385d01c,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14,f786e92e-7ac1-4035-9644-de30b8d8b6a5,Biotechnology,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [456]:
for column in data_tec.columns[2:]:
    data_tec.rename(columns={column: column[16:]}, inplace=True)

In [457]:
data_tec.head()

,id,technology_name,AR and VR,Artificial Intelligence,Biotechnology,BlockChain,Hardware,Science and Engineering,Software,Sustainability,nan
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Science and Engineering,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Hardware,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,59b8a226-eb24-4e7b-851d-924e0385d01c,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14,f786e92e-7ac1-4035-9644-de30b8d8b6a5,Biotechnology,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [458]:
data_tec = data_tec.groupby('id').max()
data_tec.head()

,technology_name,AR and VR,Artificial Intelligence,Biotechnology,BlockChain,Hardware,Science and Engineering,Software,Sustainability,nan
id,,,,,,,,,,
00003f34-ac89-4090-b592-30f2b70ff5a4,Sustainability,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0003088d-8c0b-46f6-910a-6dea65777ac4,Software,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0004d27e-6d98-4f29-a708-b989e3e38f81,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [459]:
data_tec.reset_index(inplace=True)
data_tec.drop(columns=['nan', 'technology_name'], inplace=True)

In [460]:
data_tec.shape

(56667, 9)

## `funding_round` encoding

In [418]:
data.columns

Index(['id', 'name', 'website', 'short_description', 'ipo_status',
       'founded_on', 'went_public_on', 'exited_on', 'num_funding_rounds',
       'last_equity_funding_type', 'last_equity_funding_total',
       'last_funding_at', 'headquartersCountry', 'headquartersRegion',
       'employeeCount', 'industry_name', 'technology_name', 'announcedOn',
       'moneyRaised'],
      dtype='object')

In [419]:
data_fun = data[['id', 'announcedOn', 'moneyRaised']].drop_duplicates()

In [420]:
data_fun.head()

,id,announcedOn,moneyRaised
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,2018-05-30 00:00:00.000,NaN
8,441e5d33-140e-40c6-ab5d-35f65ab8feed,2022-11-30 00:00:00.000,"{""amount"": 5000000, ""currency"": ""USD"", ""amount..."
9,441e5d33-140e-40c6-ab5d-35f65ab8feed,2021-06-17 00:00:00.000,NaN
10,59b8a226-eb24-4e7b-851d-924e0385d01c,2016-07-11 00:00:00.000,"{""amount"": 50000000, ""currency"": ""USD"", ""amoun..."
14,f786e92e-7ac1-4035-9644-de30b8d8b6a5,2018-08-01 00:00:00.000,NaN


In [421]:
data_fun['announcedOn'] = pd.to_datetime(data_fun['announcedOn'])
data_fun.sort_values(by=['id', 'announcedOn'], inplace=True)
data_fun.head(10)

,id,announcedOn,moneyRaised
120250,00003f34-ac89-4090-b592-30f2b70ff5a4,2018-08-15,"{""amount"": 15098200, ""currency"": ""USD"", ""amoun..."
54720,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,2018-07-09,"{""amount"": 11814500, ""currency"": ""USD"", ""amoun..."
54719,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,2020-03-27,"{""amount"": 27167600, ""currency"": ""USD"", ""amoun..."
79756,0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,2022-10-31,"{""amount"": 5000000, ""currency"": ""USD"", ""amount..."
253767,0003088d-8c0b-46f6-910a-6dea65777ac4,2018-05-15,"{""amount"": 2804700, ""currency"": ""USD"", ""amount..."
303969,0004d27e-6d98-4f29-a708-b989e3e38f81,2017-12-01,NaN
31069,0004e479-1be1-49d9-b54d-e6c2085dc18e,2020-01-01,"{""amount"": 22500000, ""currency"": ""USD"", ""amoun..."
31070,0004e479-1be1-49d9-b54d-e6c2085dc18e,2020-05-15,"{""amount"": 2500000, ""currency"": ""USD"", ""amount..."
31071,0004e479-1be1-49d9-b54d-e6c2085dc18e,2021-06-15,"{""amount"": 25000000, ""currency"": ""USD"", ""amoun..."
31072,0004e479-1be1-49d9-b54d-e6c2085dc18e,2021-06-18,"{""amount"": 5000000, ""currency"": ""USD"", ""amount..."


In [422]:
from preprocessing.format_moneyRaised import format_moneyRaised

format_moneyRaised(data_fun)

,id,announcedOn,moneyRaised,moneyRaised_USD
120250,00003f34-ac89-4090-b592-30f2b70ff5a4,2018-08-15,"{""amount"": 15098200, ""currency"": ""USD"", ""amoun...",15098200.0
54720,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,2018-07-09,"{""amount"": 11814500, ""currency"": ""USD"", ""amoun...",11814500.0
54719,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,2020-03-27,"{""amount"": 27167600, ""currency"": ""USD"", ""amoun...",27167600.0
79756,0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,2022-10-31,"{""amount"": 5000000, ""currency"": ""USD"", ""amount...",5000000.0
253767,0003088d-8c0b-46f6-910a-6dea65777ac4,2018-05-15,"{""amount"": 2804700, ""currency"": ""USD"", ""amount...",2804700.0
...,...,...,...,...
296908,fffe3abf-f785-4876-9281-aa45cd70a180,2016-05-01,"{""amount"": 46340300, ""currency"": ""USD"", ""amoun...",46340300.0
296907,fffe3abf-f785-4876-9281-aa45cd70a180,2016-12-01,"{""amount"": 116184300, ""currency"": ""USD"", ""amou...",116184300.0
74457,ffff528c-dd2d-489a-b6ca-1956a526b80a,2020-01-02,"{""amount"": 300000000, ""currency"": ""USD"", ""amou...",300000000.0
306647,ffffa984-a16c-4942-a968-1c5806641c30,2018-06-01,"{""amount"": 12000000, ""currency"": ""USD"", ""amoun...",12000000.0


In [423]:
data_fun['moneyRaised_USD'] = data_fun['moneyRaised_USD']/100

In [424]:
data_fun

,id,announcedOn,moneyRaised,moneyRaised_USD
120250,00003f34-ac89-4090-b592-30f2b70ff5a4,2018-08-15,"{""amount"": 15098200, ""currency"": ""USD"", ""amoun...",150982.0
54720,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,2018-07-09,"{""amount"": 11814500, ""currency"": ""USD"", ""amoun...",118145.0
54719,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,2020-03-27,"{""amount"": 27167600, ""currency"": ""USD"", ""amoun...",271676.0
79756,0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,2022-10-31,"{""amount"": 5000000, ""currency"": ""USD"", ""amount...",50000.0
253767,0003088d-8c0b-46f6-910a-6dea65777ac4,2018-05-15,"{""amount"": 2804700, ""currency"": ""USD"", ""amount...",28047.0
...,...,...,...,...
296908,fffe3abf-f785-4876-9281-aa45cd70a180,2016-05-01,"{""amount"": 46340300, ""currency"": ""USD"", ""amoun...",463403.0
296907,fffe3abf-f785-4876-9281-aa45cd70a180,2016-12-01,"{""amount"": 116184300, ""currency"": ""USD"", ""amou...",1161843.0
74457,ffff528c-dd2d-489a-b6ca-1956a526b80a,2020-01-02,"{""amount"": 300000000, ""currency"": ""USD"", ""amou...",3000000.0
306647,ffffa984-a16c-4942-a968-1c5806641c30,2018-06-01,"{""amount"": 12000000, ""currency"": ""USD"", ""amoun...",120000.0


In [425]:
data_fun.drop(columns=['moneyRaised', 'announcedOn'], inplace=True)
data_fun = data_fun[data_fun['moneyRaised_USD'] != 0]
data_fun.head(20)

,id,moneyRaised_USD
120250,00003f34-ac89-4090-b592-30f2b70ff5a4,150982.0
54720,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,118145.0
54719,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,271676.0
79756,0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,50000.0
253767,0003088d-8c0b-46f6-910a-6dea65777ac4,28047.0
31069,0004e479-1be1-49d9-b54d-e6c2085dc18e,225000.0
31070,0004e479-1be1-49d9-b54d-e6c2085dc18e,25000.0
31071,0004e479-1be1-49d9-b54d-e6c2085dc18e,250000.0
31072,0004e479-1be1-49d9-b54d-e6c2085dc18e,50000.0
31074,0004e479-1be1-49d9-b54d-e6c2085dc18e,2200000.0


In [426]:
data_fun['index'] = data_fun.groupby('id').cumcount() + 1
# pivot the data to create the desired output
output = data_fun.pivot(index='id', columns='index', values='moneyRaised_USD')
# rename the columns to match the desired output
output.columns = [str(i) for i in output.columns]
data_fun = output

/var/folders/rt/gch51dj925v29nlt01bkgkg80000gn/T/ipykernel_71457/479812745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fun['index'] = data_fun.groupby('id').cumcount() + 1


In [427]:
data_fun.reset_index(inplace=True)
data_fun = data_fun.iloc[:, 0:6]
data_fun

,id,1,2,3,4,5
0,00003f34-ac89-4090-b592-30f2b70ff5a4,150982.0,NaN,NaN,NaN,NaN
1,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,118145.0,271676.0,NaN,NaN,NaN
2,0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,50000.0,NaN,NaN,NaN,NaN
3,0003088d-8c0b-46f6-910a-6dea65777ac4,28047.0,NaN,NaN,NaN,NaN
4,0004e479-1be1-49d9-b54d-e6c2085dc18e,225000.0,25000.0,250000.0,50000.0,2200000.0
...,...,...,...,...,...,...
42699,fffd4279-32ff-46d3-8b63-58eb1aead505,250000.0,NaN,NaN,NaN,NaN
42700,fffdd79a-655f-4d82-b09a-68e81d98ef90,131207.0,333078.0,NaN,NaN,NaN
42701,fffe3abf-f785-4876-9281-aa45cd70a180,463403.0,1161843.0,NaN,NaN,NaN
42702,ffff528c-dd2d-489a-b6ca-1956a526b80a,3000000.0,NaN,NaN,NaN,NaN


In [428]:
for i in range(1, 6):
    data_fun.rename(columns={str(i): f'Round {str(i)}'}, inplace=True)

data_fun

,id,Round 1,Round 2,Round 3,Round 4,Round 5
0,00003f34-ac89-4090-b592-30f2b70ff5a4,150982.0,NaN,NaN,NaN,NaN
1,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,118145.0,271676.0,NaN,NaN,NaN
2,0001b1aa-4a8b-4ee2-b4c0-6bce0318c2c1,50000.0,NaN,NaN,NaN,NaN
3,0003088d-8c0b-46f6-910a-6dea65777ac4,28047.0,NaN,NaN,NaN,NaN
4,0004e479-1be1-49d9-b54d-e6c2085dc18e,225000.0,25000.0,250000.0,50000.0,2200000.0
...,...,...,...,...,...,...
42699,fffd4279-32ff-46d3-8b63-58eb1aead505,250000.0,NaN,NaN,NaN,NaN
42700,fffdd79a-655f-4d82-b09a-68e81d98ef90,131207.0,333078.0,NaN,NaN,NaN
42701,fffe3abf-f785-4876-9281-aa45cd70a180,463403.0,1161843.0,NaN,NaN,NaN
42702,ffff528c-dd2d-489a-b6ca-1956a526b80a,3000000.0,NaN,NaN,NaN,NaN


## Join `data_ind`, `data_tec`, and `data_fun` to original dataset

In [463]:
data.columns

Index(['id', 'name', 'website', 'short_description', 'ipo_status',
       'founded_on', 'went_public_on', 'exited_on', 'num_funding_rounds',
       'last_equity_funding_type', 'last_equity_funding_total',
       'last_funding_at', 'headquartersCountry', 'headquartersRegion',
       'employeeCount', 'industry_name', 'technology_name', 'announcedOn',
       'moneyRaised'],
      dtype='object')

In [464]:
data_merged = data.drop(columns=['industry_name',
                                 'technology_name',
                                 'announcedOn',
                                 'moneyRaised'])

In [465]:
data_merged = data_merged.drop_duplicates().reset_index(drop=True)

In [466]:
data_merged.shape

(56667, 15)

### Merging Industry:

In [469]:
data_ind.shape

(56667, 41)

In [470]:
data_ind.head(2)

,id,Advertising,Agriculture and Farming,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Computer Hardware,Consumer Electronics,Consumer Goods,Content and Publishing,...,Payments,Privacy and Security,Professional Services,Real Estate and Construction,Sales and Marketing,Software,Sports,Transportation,Travel and Tourism,Video
0,00003f34-ac89-4090-b592-30f2b70ff5a4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [471]:
data_merged_ind = data_merged.merge(data_ind, on='id', how='left')
data_merged_ind.shape

(56667, 55)

### Merging Technology:

In [472]:
data_tec.shape

(56667, 9)

In [473]:
data_tec.head(2)

,id,AR and VR,Artificial Intelligence,Biotechnology,BlockChain,Hardware,Science and Engineering,Software,Sustainability
0,00003f34-ac89-4090-b592-30f2b70ff5a4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [474]:
data_merged_tec = data_merged_ind.merge(data_tec, on='id', how='left')
data_merged_tec.shape

(56667, 63)

### Merging Funding Rounds:

In [476]:
data_fun.shape

(42704, 6)

In [477]:
data_fun.head(2)

,id,Round 1,Round 2,Round 3,Round 4,Round 5
0,00003f34-ac89-4090-b592-30f2b70ff5a4,150982.0,NaN,NaN,NaN,NaN
1,00014e45-238c-4ce1-a9d1-de7c6d5bbf2c,118145.0,271676.0,NaN,NaN,NaN


In [478]:
data_merged_all = data_merged_tec.merge(data_fun, on='id', how='left')
data_merged_all.shape

(56667, 68)

# Final Dataset post-duplicates

In [479]:
data_merged_all

,id,name,website,short_description,ipo_status,founded_on,went_public_on,exited_on,num_funding_rounds,last_equity_funding_type,...,BlockChain,Hardware,Science and Engineering,Software_y,Sustainability,Round 1,Round 2,Round 3,Round 4,Round 5
0,75139f4c-6d9d-4a03-8b72-61f73f7be74a,Youcan Robot,https://www.youcanrobot.com/,Youcan Robot is an integrated company in the w...,private,2016-01-01,NaN,NaN,1,seed,...,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,441e5d33-140e-40c6-ab5d-35f65ab8feed,Make it Home Safe,https://makeithomesafe.com/,Make it Home Safe is a mobile app that provide...,private,2017-01-01,NaN,NaN,2,seed,...,0.0,0.0,0.0,1.0,0.0,50000.0,NaN,NaN,NaN,NaN
2,59b8a226-eb24-4e7b-851d-924e0385d01c,Biveo,https://biveo.com/,Business Video Production Directory and Guide,private,2016-07-11,NaN,NaN,1,seed,...,0.0,0.0,0.0,1.0,0.0,500000.0,NaN,NaN,NaN,NaN
3,f786e92e-7ac1-4035-9644-de30b8d8b6a5,Spaced-X,http://www.spaced-x.com/,Medical grade consumption accessories for the ...,private,2018-02-24,NaN,NaN,1,pre_seed,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,45fff4cb-2c30-4fef-b8cd-c18239c9e6a0,Superflex,http://superflextechnologies.com,Apparel enhancing muscle strength targeting el...,private,2016-02-01,NaN,NaN,2,NaN,...,0.0,0.0,0.0,0.0,0.0,9600000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56662,86e5c2b7-1508-4bb6-b6b0-92269a93d0c6,TradeWindow,https://tradewindow.io/,TradeWindow is a global trade platform for exp...,private,2018-12-03,NaN,NaN,3,series_b,...,0.0,0.0,0.0,1.0,0.0,1759743.0,4807185.0,10484598.0,NaN,NaN
56663,ed1d84d8-4072-459a-80ed-a18ec7a13513,Alcide,http://www.alcide.io,Alcide provides a cloud-native security platfo...,private,2016-01-01,NaN,2021-01-31,4,series_a,...,0.0,0.0,0.0,1.0,0.0,100000.0,5200000.0,7000000.0,NaN,NaN
56664,e3e6968e-0bd8-4fa8-b529-8fe87d208be9,Anvert Kft,https://anvert.com/,Anvert is a web-based app that reduces the tim...,private,2018-01-01,NaN,NaN,4,seed,...,0.0,0.0,0.0,1.0,0.0,34733.0,112321.0,17998.0,404398.0,NaN
56665,2b2923fe-132d-4167-a97f-682e575df22f,Drōv Technologies,https://www.drovtechnologies.com/,Drōv is a technology company developing innova...,private,2016-01-01,NaN,NaN,1,seed,...,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN


In [483]:
data_merged_all.describe()

,num_funding_rounds,employeeCount,Advertising,Agriculture and Farming,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Computer Hardware,Consumer Electronics,Consumer Goods,...,BlockChain,Hardware,Science and Engineering,Software_y,Sustainability,Round 1,Round 2,Round 3,Round 4,Round 5
count,56667.000000,52234.000000,56667.000000,56667.000000,56667.000000,56667.000000,56667.00000,56667.000000,56667.000000,56667.000000,...,56667.000000,56667.000000,56667.000000,56667.000000,56667.000000,4.270400e+04,1.990500e+04,1.088300e+04,6.201000e+03,3.560000e+03
mean,2.001941,57.874201,0.028094,0.026806,0.019341,0.105758,0.02467,0.137311,0.069811,0.014576,...,0.048547,0.067270,0.026576,0.643743,0.060211,7.603390e+06,9.656692e+06,1.334077e+07,1.749175e+07,2.202715e+07
std,1.572155,449.148989,0.165243,0.161517,0.137722,0.307531,0.15512,0.344178,0.254831,0.119851,...,0.214920,0.250492,0.160843,0.478897,0.237880,1.171411e+08,8.165809e+07,5.725938e+07,5.928507e+07,7.050248e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,1.000000e+00,2.750000e+02,2.770000e+02,3.730000e+02
25%,1.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.396152e+05,2.650000e+05,4.169455e+05,5.650000e+05,6.500000e+05
50%,1.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,5.652885e+05,1.295000e+06,2.000000e+06,2.722234e+06,3.346794e+06
75%,2.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,2.430765e+06,5.000000e+06,8.399292e+06,1.200000e+07,1.553188e+07
max,22.000000,32098.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.660000e+10,9.049262e+09,2.262315e+09,2.600000e+09,2.400000e+09


In [484]:
data_merged_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56667 entries, 0 to 56666
Data columns (total 68 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                56667 non-null  object 
 1   name                              56667 non-null  object 
 2   website                           55204 non-null  object 
 3   short_description                 56665 non-null  object 
 4   ipo_status                        56667 non-null  object 
 5   founded_on                        56667 non-null  object 
 6   went_public_on                    664 non-null    object 
 7   exited_on                         3325 non-null   object 
 8   num_funding_rounds                56667 non-null  int64  
 9   last_equity_funding_type          48618 non-null  object 
 10  last_equity_funding_total         35500 non-null  object 
 11  last_funding_at                   56667 non-null  object 
 12  head